In [1]:
#cleaned dataset for: 

#location: Austin
#category: Restaurant 
#open/close: only open restaurants 
#fast food: removal of fast food/chain restaurants, BUT include locations with "Breakfast & Brunch"

#also included a merge between check_in data and business data 

In [2]:
import json
import pandas as pd

In [3]:
business_data = pd.read_json('yelp_academic_dataset_business.json',lines=True)
#len(business_data)
#business_data.columns
#business_data.head()

In [4]:
business_data[business_data['business_id']=='6cIzTqQGthBctMWzxLHwRQ']

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
37063,6cIzTqQGthBctMWzxLHwRQ,Nervous Charlie's,"5501 N Lamar Blvd, Ste B101",Austin,TX,78751,30.32294,-97.727974,4.5,332,1,"{'GoodForKids': 'True', 'GoodForMeal': '{'dess...","Restaurants, Sandwiches, Bagels, Breakfast & B...","{'Monday': '7:0-15:0', 'Tuesday': '7:0-15:0', ..."


In [5]:
austin_business_data = business_data[business_data['city']=='Austin']
#filter Yelp dataset based ONLY on Austin establishments 

In [6]:
arbd= austin_business_data[austin_business_data['categories'].str.contains('Restaurant',na=False)]

In [7]:
#len(austin_restaurants_business_data)
#check for length of new dataset, will primarily be using this moving forward 
#fits criteria for 1)reviews located in Austin + 2)reviews only by restaurant category

In [8]:
#austin_restaurants_business_data.to_excel('C:\\Users\\blin3\\Machine Learning\\WiDS\\covid_19_dataset_2020_06_10\\rest_bus_data_x.xlsx',index=False)

#transferred to excel for a better outlook on data

#arbd = pd.read_excel('rest_bus_data_x.xlsx')
##NOTE: NEED TO RUN THE previous cell to save the original arbd dataset as a new excel file 
#Just redefining the df b/c I want to, you can easily just call the original df as "arbd" if you desire

#arbd = austin restaurant business data 

In [9]:
#merge checkin dataset with business dataset 

In [10]:
check_in = pd.read_json('yelp_academic_dataset_checkin.json',lines=True)

In [11]:
check_in[check_in['business_id']=='6cIzTqQGthBctMWzxLHwRQ']

,business_id,date
16478,6cIzTqQGthBctMWzxLHwRQ,"2018-09-20 00:07:21, 2018-09-21 12:15:54, 2018..."


In [12]:
arbdc = pd.merge(arbd,check_in, on='business_id')
#arbd dataset + check in dataset 

In [13]:
arbdc['is_open'].value_counts()
#2796 restaurants are open, 1988 are not open

1    2796
0    1988
Name: is_open, dtype: int64

In [14]:
arbdc = arbdc[arbdc['is_open']==1]
#only open businesses 4784 -->2796

In [15]:
arbdc = arbdc.drop(['city', 'state','latitude','longitude'], axis=1)

In [16]:
l1 = arbdc['name'].value_counts()[:30].index.tolist()
#top 31 general locations 
#.index.tolist() returns the list of values WITHOUT the actual values 
#if you just did: list(df['name'].value_counts()[:30]), then you would get a list of #'s, like [27,26,25....], instead of
#["Wendy's", "McDonald's",.....]

In [17]:
l1

['Subway',
 "McDonald's",
 'Sonic Drive-In',
 "Domino's Pizza",
 'Taco Bell',
 'Jack in the Box',
 "Wendy's",
 'Whataburger',
 "P. Terry's Burger Stand",
 'Chipotle Mexican Grill',
 "Jimmy John's",
 'JuiceLand',
 'Thundercloud Subs',
 "Papa John's Pizza",
 'Pizza Hut',
 'Chick-fil-A',
 'Taco Cabana',
 "Jersey Mike's Subs",
 'Panda Express',
 'Starbucks',
 'Popeyes Louisiana Kitchen',
 'Burger King',
 'El Pollo Rico',
 'Torchys Tacos',
 'KFC',
 'Wingstop',
 "Chi'Lantro",
 'IHOP',
 'Tacodeli',
 'Dairy Queen']

In [18]:
l2 = arbdc[arbdc['categories'].str.contains('Fast Food',na=False)]['name'].value_counts()[:30].index.tolist()
#top31 locations with 'fast food' in its category description 
#31 is an abritrary number 

In [19]:
l3 = [x for x in l1 if x not in l2]  #all the items that exist in l1, that ARE NOT in l2
l3

["Domino's Pizza",
 'JuiceLand',
 'Thundercloud Subs',
 "Papa John's Pizza",
 'Pizza Hut',
 'Starbucks',
 'El Pollo Rico',
 'Torchys Tacos',
 'Wingstop',
 "Chi'Lantro",
 'IHOP',
 'Tacodeli']

In [20]:
l2.extend(l3) #use extend instead of append, otherwise the l3 will be a nested list

In [21]:
#list(df[df['categories'].str.contains('Fast Food',na=False)]['name'].unique())[0:19]
#fast_food= list(df[df['categories'].str.contains('Fast Food',na=False)]['name'].unique())[0:19]    

#^^initial lines of code to remove fast food restaurants, but did not contain 
# 1) the most frequent # of fast food restaurants; didn't use the value_counts attribute 
# 2) did not include other chain stores that did not have the fast food category label; ex: SBUX, Pizza Hut

In [22]:
bfast_check_df = arbdc[arbdc['name'].isin(l2)]   
#creating a new df that ONLY contains the locations in l2

In [23]:
bfast_check_df[bfast_check_df['categories'].str.contains('Breakfast | Diner | Bagel',na=False)].name.value_counts()

Taco Bell                  19
Jack in the Box            19
IHOP                        7
Torchys Tacos               7
Tacodeli                    6
Church's Chicken            4
Taco Cabana                 2
Starbucks                   2
Chick-fil-A                 1
Burger King                 1
P. Terry's Burger Stand     1
Name: name, dtype: int64

In [24]:
l2.remove('IHOP')

In [25]:
'IHOP' in l2  #check if IHOP is in l2
#https://stackoverflow.com/questions/7571635/fastest-way-to-check-if-a-value-exists-in-a-list

False

In [26]:
#arbdc[arbdc['categories'].str.contains('Diners',na=False)].name.value_counts()
#additional check for restaurants with "diner" label, potential competition 

In [27]:
#pd.set_option('display.max_colwidth', None)
#arbdc[arbdc['name']=='IHOP'].categories
#check to see how categories can be different for the same restaurant
#ex: IHOP 109 contains "Diners", but IHOP 1455 does not

In [28]:
arbdc = arbdc[~arbdc['name'].isin(l2)]   
#removing every row that has a name in the l2 list 
#the (~) means "not" 

In [29]:
arbdc[arbdc['name']=="Wendy's"]
#post-check to see if Wendy's is in the new df 

,business_id,name,address,postal_code,stars,review_count,is_open,attributes,categories,hours,date


In [30]:
len(arbdc)

2384

In [31]:
arbdc = arbdc.groupby('postal_code').filter(lambda x: len(x)>1)
#removing zip codes with only 1 restaurant in them

In [32]:
len(arbdc.groupby('postal_code').filter(lambda x: len(x)>1))
#https://stackoverflow.com/questions/49735683/python-removing-rows-on-count-condition
#9 zip codes removed 

2375

In [33]:
len(arbdc[arbdc['postal_code']=='78798'])

6

In [34]:
extra_zips = ['78617','78798']
#removing extra zip codes as they are irrelevant to the histogram shown below / skew results 

In [35]:
for i in extra_zips: 
    x= arbdc[arbdc['postal_code'] == i].index
    arbdc= arbdc.drop(x)
#removal of the rows that contain the zip codes 78617, 78798

In [36]:
arbdc['postal_code'].value_counts().tail()
#check value counts again, make sure that no zip codes have only 1 restaurant

78736    8
78712    6
78747    6
78733    5
78739    3
Name: postal_code, dtype: int64

In [37]:
arbdc[arbdc['postal_code']=='78617']

,business_id,name,address,postal_code,stars,review_count,is_open,attributes,categories,hours,date


In [38]:
arbdc.to_excel('Yelp_Update_Cleaned_5_20.xlsx')

In [40]:
test = check_in.iloc[0]

In [49]:
x = check_in.iloc[0].business_id
x

'--0r8K_AQ4FZfLsX3ZYRDA'

In [43]:
business_data.business_id

0         6iYb2HFDywm3zjuRg0shjw
1         tCbdrRPZA0oiIYSmHG3J0w
2         bvN78flM8NLprQ1a1y5dRg
3         oaepsyvc0J17qwi8cfrOWg
4         PE9uqAjdw0E4-8mjGl3wVA
                   ...          
160580    D2mHoIDXx9N8mS1pGoKV9Q
160581    bQX-kwVTyZgcdZGEPzce6Q
160582    wvFZ06nmPmQ2-IVoPqVYLA
160583    GB75wPibj3IjNauaoCxyGA
160584    ngmLL5Y5OT-bYHKU0kKrYA
Name: business_id, Length: 160585, dtype: object

In [69]:
y= business_data[business_data['business_id']==x]['city'].astype(str).values[0]
y


'Columbus'

In [65]:
business_data['test']=''

In [73]:
i=0
while i< len(business_data):
    x=check_in.iloc[i]
    if x == business_data['business_id'].iloc[i]:
        y = business_data[business_data['business_id']==x]['city'].astype(str)

        business_data = business_data.loc[i,'test']=y    
    i+=1

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [46]:
business_data['business_id'].iloc[0]

'6iYb2HFDywm3zjuRg0shjw'

In [72]:
business_data[business_data['business_id']=='--0r8K_AQ4FZfLsX3ZYRDA'].review_count

29957    5
Name: review_count, dtype: int64